## Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum

## Class Definition 

In [2]:
class ServerIDs(Enum):
    """
    Description: 
        Class storing server names
    """
    msn = 'MSN'
    asn_1 = 'ASN1'
    asn_2 = 'ASN2'

class Groups(Enum):
    """
    Description: 
        Class storing group ids
    """
    Group_1 = 1
    Group_2 = 2
    Group_3 = 3

    
    
class Status(Enum):
    """
    Description: 
        Class storing status - First Come First Serve (FCFS) principle
    """
    handled = -1
    served = 0
    arrived = np.inf
    
class CustomerGroup:
    """
    Description: 
        Class storing all customer group specific information
    """
    def __init__(self,ID,popularity,activity_pattern,distances):
        self.ID = ID
        self.popularity = popularity
        self.weights=popularity/np.sum(popularity)
        self.activity_pattern = activity_pattern
        self.distances = distances
    
    def best_server_options(self):
        """
        Description: 
            Finding server options
        Return:
            list (str) - sorted list of server options from best to worst
        """
        return sorted(self.distances, key=self.distances.get)
     
class Server:
    """ 
    Description: 
        Class representing the servers {MSN, ASN1, ASN2}
    """
    def __init__(self, id_, movies_stored, capacity, movie_sizes):
        self.id = id_
        self.movies_stored = movies_stored
        self.capacity = capacity
        self.movie_sizes = movie_sizes
        
         # Parameter for checking that the capacity limited of the server is not succeeded
        self.capacity_check = capacity
        self.check_capacity_limit()
        
    def check_capacity_limit(self):
        """
        Description: 
            Check capacity limited is not succeeded
        """
        for movie in self.movies_stored:
            self.capacity_check -= self.movie_sizes[movie]
            if self.capacity_check < 0:
                raise Exception('Server Capacity Exceeded')
        
class LoadBalancer:
    """ 
    Description: 
        Superclass storing the servers and all associated information (e.g. such as serving time)
    """
    def __init__(self, movies_stored_msn, movies_stored_asn_1, movies_stored_asn_2,\
                 capacities, serve_times, movie_sizes):
        self.msn = Server(ServerIDs.msn.value, movies_stored_msn, capacities[0], movie_sizes)
        self.asn_1 = Server(ServerIDs.asn_1.value, movies_stored_asn_1, capacities[1], movie_sizes)
        self.asn_2 = Server(ServerIDs.asn_2.value, movies_stored_asn_2, capacities[2], movie_sizes)
        self.serve_times = serve_times
        self.movie_sizes = movie_sizes
        
    def get_serve_time(self, movie: int, group_id: int, server_id: str):
        """
        Description: 
            Return service time based on input arguments
        Args: 
            int - chosen movie
            int - group_id of customer group \in {1,2,3}
            string - server_id of server \in {'MSN', 'ASN1', 'ASN2'}
        Return:
            Int - serve time
        """
        movie_size = self.movie_sizes[movie]
        if movie_size < 900:
            return self.serve_times[900][server_id][group_id]
            
        elif movie_size >= 900 and movie_size < 1100:
            return self.serve_times[1100][server_id][group_id]
            
        elif movie_size >= 1100:
            return self.serve_times[1500][server_id][group_id]
        else:
            raise Exception(f'Movie Size: {movie_size}')
        
class SingleCustomer:
    """ 
    Description: 
        Class representing single customer
    """
    def __init__(self,id_,time,movie_choice,server_address,waiting_time):
        self.id_ = id_
        self.status = Status.arrived.value
        self.time = time
        self.movie_choice = movie_choice
        self.waiting_time = waiting_time
        self.server_address = server_address

In [3]:
class Scenario:
    
    def __init__(self, asn_allocations_1: list, asn_allocations_2: list):
        
        self.capacities = [np.inf, 3500,3500]

        self.movie_sizes = {
            0: 850,
            1: 950,
            2: 1000,
            3: 1200,
            4: 800,
            5: 900,
            6: 1000,
            7: 750,
            8: 700,
            9: 1100
        }

        self.movies_stored_msn = list(range(0, 10))
        self.movies_stored_asn_1 = asn_allocations_1
        self.movies_stored_asn_2 = asn_allocations_2

        # --- Table 2 ---
        # Preferences
        self.popularities = [[2,4,9,8,1,3,5,7,10,6],[6,1,3,4,7,9,2,5,8,10],[4,7,3,6,1,10,2,9,8,5]]

        # --- Table 3 ---
        # Lambdas
        self.activity_patterns = [[0.8,1.2,0.5],[0.9,1.3,0.3],[0.7,1.5,0.4]]

        # --- Table 4 ---
        self.distances_g1 = {ServerIDs.msn.value: 0.5, ServerIDs.asn_1.value: 0.2, ServerIDs.asn_2.value: np.inf}
        self.distances_g2 = {ServerIDs.msn.value: 0.5, ServerIDs.asn_1.value: 0.3, ServerIDs.asn_2.value: 0.4}
        self.distances_g3 = {ServerIDs.msn.value: 0.5, ServerIDs.asn_1.value: np.inf, ServerIDs.asn_2.value: 0.2}

        # --- Table 5 ---
        # [700- 900)
        self.msn_serve_time_900 = {1: 9, 2: 8, 3: 10}
        self.asn_1_serve_time_900 = {1: 3, 2: 4, 3: np.inf}
        self.asn_2_serve_time_900 = {1: np.inf, 2: 5, 3: 4}

        self.serve_times_900 = {
            ServerIDs.msn.value: self.msn_serve_time_900,
            ServerIDs.asn_1.value: self.asn_1_serve_time_900,
            ServerIDs.asn_2.value: self.asn_2_serve_time_900
        }

        # [900- 1100)
        self.msn_serve_time_1100 = {1: 12, 2: 11, 3: 13}
        self.asn_1_serve_time_1100 = {1: 4, 2: 5, 3: np.inf}
        self.asn_2_serve_time_1100 = {1: np.inf, 2: 6, 3: 5}

        self.serve_times_1100 = {
            ServerIDs.msn.value: self.msn_serve_time_1100,
            ServerIDs.asn_1.value: self.asn_1_serve_time_1100,
            ServerIDs.asn_2.value: self.asn_2_serve_time_1100
        }

        # [1100- 1500)
        self.msn_serve_time_1500 = {1: 15, 2: 14, 3: 16}
        self.asn_1_serve_time_1500 = {1: 5, 2: 6, 3: np.inf}
        self.asn_2_serve_time_1500 = {1: np.inf, 2: 7, 3: 6}

        self.serve_times_1500 = {
            ServerIDs.msn.value: self.msn_serve_time_1500,
            ServerIDs.asn_1.value: self.asn_1_serve_time_1500,
            ServerIDs.asn_2.value: self.asn_2_serve_time_1500
        }

        self.serve_times = {
            900: self.serve_times_900,
            1100: self.serve_times_1100,
            1500: self.serve_times_1500
        }

## Function Definiton

In [4]:
def exponential_rng(lam=1.0, u=None):  
    """ 
    Description:
        Generates exponential random number
    Args:
        float - lam, the rate parameter, the inverse expectation of the distribution
    Return:
        float - Exponential random number with given rate
    """
    if not u:
        return -np.log(np.random.rand()) / lam
    else:
        return -np.log(u) / lam

def homogeneous_poisson_process(lam, T):
    """ 
    Description:
        Simulate arrivals via homogeneous Poisson process
    Args:
        float - lam, the rate parameter, the inverse expectation of the distribution
        int - simulation period
    Return:
        list (float) - Arrival times
    """
    arrival_times=[]
    curr_arrival_time=0
    while True:
        curr_arrival_time += exponential_rng(lam)
        if curr_arrival_time > T:
            break
        else:
            arrival_times.append(curr_arrival_time)
    return arrival_times


def homogeneous_poisson_process_variance_reduction(lam, T, u):
    """ 
    Description:
        Simulate arrivals via homogeneous Poisson process
    Args:
        float - lam, the rate parameter, the inverse expectation of the distribution
        int - simulation period
    Return:
        list (float) - Arrival times
    """
    arrival_times=[]
    curr_arrival_time=0
    counter = -1
    while True:
        counter += 1
        curr_arrival_time += exponential_rng(lam, u[counter])
        if curr_arrival_time > T:
            break
        else:
            arrival_times.append(curr_arrival_time)
    return arrival_times



def adjust_time_and_pick_a_movie(arrival_times: list, delta_T: int, G: CustomerGroup, load_balancer: LoadBalancer)-> list:
    """
    Description:
        Create single customers and assign attributes (e.g. movie, server address)
    Args:
        list (float) - arrival times
        int - delta_T, time offset for arrival times
        class - Customer group
        class - Load balancer
    Return:
        list (class) - List of customers
    """
    output = []
    for arrival_time in arrival_times:
        movie = np.random.choice(np.arange(10, step=1), size=None, replace=True, p=G.weights)
        servers_with_movie = check_movie_availability_on_server(movie,load_balancer)
        best_servers = G.best_server_options()
        serverAddress = assign_server(servers_with_movie,best_servers)
        waiting_time = G.distances[serverAddress]
        output.append(SingleCustomer(G.ID, arrival_time + delta_T+waiting_time,\
                                     movie,serverAddress,waiting_time))
    return output

def assign_server(servers_with_movie,best_servers):
    """
    Description:
        Find server assignment for single customer
    Args:
        list (str) - server names that movie is stored on
        list (str) - sorted list of optimal servers
    Return:
        list (class) - List of customers
    """
    for best in best_servers:
        if best in servers_with_movie:
            if best!=np.inf:
                return best
            else:
                raise Exception('Movie not available for customer!')

def check_movie_availability_on_server(movie: int,load_balancer: LoadBalancer):
    """
    Description:
        Find all servers that the selected movie is stored on
    Args:
        int - selected movie by customer
        class - Load balancer
    Return:
        list (str) - server names that movie is stored on
    """        
    servers_with_movie=[]
    if movie in load_balancer.msn.movies_stored:
        servers_with_movie.append(ServerIDs.msn.value)
    if movie in load_balancer.asn_1.movies_stored:
        servers_with_movie.append(ServerIDs.asn_1.value)
    if movie in load_balancer.asn_2.movies_stored:
        servers_with_movie.append(ServerIDs.asn_2.value)
    if servers_with_movie==[]:
        raise Exception('Movie not stored on any server!')
    return servers_with_movie

In [5]:
def update_and_sort_customers_queue(customers: list, current_time: float):
    """
    Description:
        Updates the time for each new customer(arrival)
    Args:
        list - customers list
        float - current_time
    Return:
        the list of sorted customers (handled customers first)
    """
    customers = sorted(customers, key = lambda customer: (customer.time, customer.status))
    for customer in customers:
        
        if customer.time <= current_time and customer.status != Status.handled.value:
            customer.waiting_time += (current_time - customer.time)
            customer.time = current_time
        else:
            assert (customer.time == current_time) and (customer.status == Status.handled.value)
            return sorted(customers, key = lambda customer: (customer.time, customer.status))

In [6]:
def generate_customers(G1: CustomerGroup, G2: CustomerGroup, G3: CustomerGroup, LB: LoadBalancer, u=np.array([])):
    """
    Description:
        Create happy hour demand from three customer groups
    Args:
        class - Customer group 1
        class - Customer group 2
        class - Customer group 3
        class - Load balancer
    Return:
        list (class) - List of customers
    """ 
    
    # Simulation time in seconds
    T = 20 * 60
    
    requests = []
    
    # Generate Arrivals per group via homogeneous Poisson process based on group specific activity pattern
    for activity_number in range(0,3):
        index = activity_number * 3
        
        # either 0, 20, 40 to make 1 hour of requests
        delta_T = activity_number * 20 * 60
        
        # Time of the events
        if not np.any(u):
            arrival_times_1 = homogeneous_poisson_process(G1.activity_pattern[activity_number], T)
            arrival_times_2 = homogeneous_poisson_process(G2.activity_pattern[activity_number], T)
            arrival_times_3 = homogeneous_poisson_process(G3.activity_pattern[activity_number], T)
        else:
            arrival_times_1 = homogeneous_poisson_process_variance_reduction(G1.activity_pattern[activity_number], T, u[0 + index,:])
            arrival_times_2 = homogeneous_poisson_process_variance_reduction(G2.activity_pattern[activity_number], T, u[1 + index,:])
            arrival_times_3 = homogeneous_poisson_process_variance_reduction(G3.activity_pattern[activity_number], T, u[2 + index,:])
            
        customers_1 = adjust_time_and_pick_a_movie(arrival_times_1, delta_T, G1, LB)
        customers_2 = adjust_time_and_pick_a_movie(arrival_times_2, delta_T, G2, LB)
        customers_3 = adjust_time_and_pick_a_movie(arrival_times_3, delta_T, G3, LB)
    
        merged_customers = customers_1 + customers_2 + customers_3
        
        merged_customers.sort(key=lambda customers:customers.time)
        
        requests = requests + merged_customers
    return requests

def process_customers(customers, load_balancer, u=None):
    """
    Description:
        Process all customers of specified server and calcuate processing times accordingly
    Args:
        list (class) - List of unserved customers
        class - Load balancer
    Return:
        list (class) - List of served customers
    """ 
    
    # First customer
    current_time = customers[0].time
    # List of served customers
    customers_served = []
    
    # either the server is busy or not
    server_busy = False
    server_busy_count = 0
    
    counter = -1
    while len(customers):
        c = customers[0]
        
        # A new request arrives to the server
        # This request has to be "handled"
        # The handle time follows an exponential distribution
        # with the mean of 0.5 second
        if c.status == Status.arrived.value and not server_busy:
            server_busy_count = 0
            
            c.status = Status.handled.value
            if np.any(u):
                counter += 1
                time_to_handle = exponential_rng(lam=2, u=u[counter])
            else:
                time_to_handle = exponential_rng(lam=2)
            
            if c.time > current_time:
                current_time = c.time

            c.waiting_time += time_to_handle
            current_time += time_to_handle
            c.time += time_to_handle
                
            server_busy = True
            #print('{:<15} {:<15}'.format(f'Time: {round(current_time,2)}s', f'Event: New Arrival'))
        
        # The client request was already handled
        # Now we have to serve the movie
        # The serve time is defined in Table 5 + some noise
        # The noise is uniformly distributed between [0.3, 0.7]
        elif c.status == Status.handled.value and server_busy:
            
        
            movie = c.movie_choice
            group_id = c.id_
            server_id = c.server_address
            time_to_serve = load_balancer.get_serve_time(movie, group_id, server_id)
            time_to_serve += np.random.uniform(0.3, 0.7)
            server_busy_count = 0
            server_busy = False
            
            c.status = Status.served.value
            c.waiting_time += time_to_serve
            customers_served.append(c)
            customers.pop(0)
            #print('{:<15} {:<15}'.format(f'Time: {round(current_time,2)}s', f'Event: Customer Served'))
        

    
        if server_busy:
            # A new request arrives but the server is busy
            # update the waiting time and time
            customers = update_and_sort_customers_queue(customers, current_time)
        else:
            customers = sorted(customers, key = lambda customer: (customer.time, customer.status))
            
    return customers_served


def handle_requests(scenario: Scenario, u=np.array([])):
    """
    Description:
        Simulate customer arrival and request handling process
    Return:
        list (class) - List of served customers
    """ 
    
    # assuming that the ASNs have the same movies stored - Otherwise adjust movies_stored_asn
    load_balancer = LoadBalancer(
        scenario.movies_stored_msn, 
        scenario.movies_stored_asn_1, 
        scenario.movies_stored_asn_2,
        scenario.capacities, 
        scenario.serve_times, 
        scenario.movie_sizes
    )
    
    G1 = CustomerGroup(
        Groups.Group_1.value,
        scenario.popularities[0],
        scenario.activity_patterns[0],
        scenario.distances_g1
    )
    G2 = CustomerGroup(
        Groups.Group_2.value,
        scenario.popularities[1],
        scenario.activity_patterns[1],
        scenario.distances_g2
    )
    G3 = CustomerGroup(
        Groups.Group_3.value,
        scenario.popularities[2],
        scenario.activity_patterns[2],
        scenario.distances_g3
    )
    
    # Generate customers according to customer groups
    customers = generate_customers(G1,G2,G3,load_balancer,u)
    
    customers_msn = []
    customers_asn_1 = []
    customers_asn_2 = []
    
    for customer in customers:
        
        if customer.server_address==ServerIDs.msn.value:
            customers_msn.append(customer)
        elif customer.server_address==ServerIDs.asn_1.value:
            customers_asn_1.append(customer)
        elif customer.server_address==ServerIDs.asn_2.value:
            customers_asn_2.append(customer)
    
    # Sorting to ensure order dependent on time
    customers_msn.sort(key=lambda customers:customers.time)
    customers_asn_1.sort(key=lambda customers:customers.time)
    customers_asn_2.sort(key=lambda customers:customers.time)
    
    
    # Get customers per server
    print('='*47)
    print('MSN')
    print('='*47)
    print('{:<30}'.format(f'MSN Customers Length: {len(customers_msn)}'))
    if np.any(u):
        customers_msn = process_customers(customers_msn, load_balancer, u[9,:])
    else:
        customers_msn = process_customers(customers_msn, load_balancer)
    print('='*47)
    print('ASN1')
    print('='*47)
    print('{:<30}'.format(f'ASN1 Customers Length: {len(customers_asn_1)}'))
    
    if np.any(u):
        customers_asn_1 = process_customers(customers_asn_1, load_balancer, u[10, :])
    else:
        customers_asn_1 = process_customers(customers_asn_1, load_balancer)
    print('='*47)
    print('ASN2')
    print('='*47)
    print('{:<30}'.format(f'ASN2 Customers Length: {len(customers_asn_2)}'))
    if np.any(u):
        customers_asn_2 = process_customers(customers_asn_2, load_balancer, u[11,:])
    else:
        customers_asn_2 = process_customers(customers_asn_2, load_balancer)
    
    # Generate all stats that you want
    return customers_msn + customers_asn_1 + customers_asn_2

def get_statistics(results: list):
    """
    Description:
        Calculate performance indicators / statistics of process
    Args:
        list (class) - List of served customers
    Return:
        dict - Statistics of process
    """ 
    waiting_times = dict()
    waiting_times[ServerIDs.msn.value] = []
    waiting_times[ServerIDs.asn_1.value] = []
    waiting_times[ServerIDs.asn_2.value] = []
    waiting_times[1] = []
    waiting_times[2] = []
    waiting_times[3] = []
    for customer in results:
        
        waiting_time = customer.waiting_time
        waiting_times[customer.server_address].append(waiting_time)    
        waiting_times[customer.id_].append(waiting_time)
    
    
    statistics = dict()
    for key_ in waiting_times.keys():
        waiting_list = np.array(waiting_times[key_])
        statistics[key_] = {
            'mean': waiting_list.mean(),
            'std': waiting_list.std(),
            'max': waiting_list.max(),
            'min': waiting_list.min(),
            'median': np.median(waiting_list),
            'q25': np.quantile(waiting_list, 0.25),
            'q75': np.quantile(waiting_list, 0.75),
            'var': np.var(waiting_list)
        }
    return statistics

## Input Definition

In [22]:
# Develop a discrete event simulation...
def test_run():
    scenario = Scenario([3,4,7,8], [3,4,7,8])
    results = handle_requests(scenario)
    statistics = get_statistics(results)
    print(statistics)
    scenario = Scenario([2,3,9], [2,3,9])
    results = handle_requests(scenario)
    statistics = get_statistics(results)
    print(statistics)
    
test_run()

MSN
MSN Customers Length: 5042    
ASN1
ASN1 Customers Length: 2708   
ASN2
ASN2 Customers Length: 1346   
{'MSN': {'mean': 49.53670934685184, 'std': 40.76545885439624, 'max': 136.16102669385756, 'min': 8.854833078341208, 'median': 30.965117463545408, 'q25': 14.67426874858784, 'q75': 86.10631992065106, 'var': 1661.8226356094733}, 'ASN1': {'mean': 5.6391078727345585, 'std': 1.3660244217734054, 'max': 14.2330363164894, 'min': 3.513662362842797, 'median': 5.416636690521022, 'q25': 4.6684651101997465, 'q75': 6.439216953117726, 'var': 1.8660227208813664}, 'ASN2': {'mean': 5.89544439835845, 'std': 1.1041205612482765, 'max': 10.305718711451348, 'min': 4.511612801551753, 'median': 5.471537783287059, 'q25': 4.981791266684579, 'q75': 6.812276755293847, 'var': 1.2190822137712092}, 1: {'mean': 26.653140539479843, 'std': 35.30433662052365, 'max': 134.78836102307517, 'min': 3.513662362842797, 'median': 12.9375849225063, 'q25': 4.856477401929646, 'q75': 24.873130167856097, 'var': 1246.396184215247}, 

In [8]:
def moving_mean_var(new_data, old_mean, old_var, t):
    """ Calculates moving sameple mean and variance at time t.
    
    Keywords:
        new_data (float): new data point arriving at time t.
        old_mean (float): previous sample mean.
        old_var (float): previous sample variance.
        t (int): time index
    
    Returns:
        new_mean (float): updated sample mean.
        new_var (float): updated sample variance.
    """
    if t == 1:
        new_mean = new_data
        new_var = 0
    else:
        new_mean = old_mean + (new_data - old_mean) / t
        new_var = (1 - 1 / (t - 1)) * old_var + t * (new_mean - old_mean)**2
    return new_mean, new_var

In [9]:
# Bootstrapping


precision = 2
t = 0

groups = [
    ServerIDs.msn.value,
    ServerIDs.asn_1.value,
    ServerIDs.asn_2.value,
    Groups.Group_1.value,
    Groups.Group_2.value,
    Groups.Group_3.value
]

final_statistics = dict()


parameter_queue = 'mean'
parameter_queue_mean = f'{parameter_queue}_queue_mean'
parameter_queue_var = f'{parameter_queue}_queue_var'
parameter_queue_mean_all = f'{parameter_queue}_queue_mean_all'
parameter_queue_var_all = f'{parameter_queue}_queue_var_all'
parameter_queue_all = f'{parameter_queue}_all'

for group in groups:
    final_statistics[group] = dict()

for group in groups:
    final_statistics[group][parameter_queue_mean] = 0
    final_statistics[group][parameter_queue_var] = 0
    final_statistics[group][parameter_queue_mean_all] = []
    final_statistics[group][parameter_queue_var_all] = []
    final_statistics[group][parameter_queue_all] = []

scenario = Scenario([2,3,9], [2,3,9])
converged = False
while not converged: 
    t += 1
    
    # Run simulation
    results = handle_requests(scenario)

    # Collect statistics
    statistics = get_statistics(results)
    
    for group in groups:
        
        par_queue = statistics[group][parameter_queue]
        mean, var = moving_mean_var(
            par_queue,\
            final_statistics[group][parameter_queue_mean],\
            final_statistics[group][parameter_queue_var],\
            t
        )
        
        final_statistics[group][parameter_queue_all].append(par_queue)
        final_statistics[group][parameter_queue_mean_all].append(mean)
        final_statistics[group][parameter_queue_var_all].append(var)
        
    # Check if necessary precision reached
    converged = True
    for group in groups:
        var = final_statistics[group][parameter_queue_var_all][-1]
        converged = converged and (t >= 100 and np.sqrt(var / t) < precision)
    print(f'{t}, {converged}, {var}')

MSN
MSN Customers Length: 6091    
ASN1
ASN1 Customers Length: 2240   
ASN2
ASN2 Customers Length: 819    
1, False, 0
MSN
MSN Customers Length: 6154    
ASN1
ASN1 Customers Length: 2225   
ASN2
ASN2 Customers Length: 799    
2, False, 8158.9946354239655
MSN
MSN Customers Length: 5974    
ASN1
ASN1 Customers Length: 2093   
ASN2
ASN2 Customers Length: 786    
3, False, 4461.186519694559
MSN
MSN Customers Length: 6145    
ASN1
ASN1 Customers Length: 2154   
ASN2
ASN2 Customers Length: 796    
4, False, 4182.7211862148115
MSN
MSN Customers Length: 6000    
ASN1
ASN1 Customers Length: 2231   
ASN2
ASN2 Customers Length: 805    
5, False, 2811.080269755307
MSN
MSN Customers Length: 6045    
ASN1
ASN1 Customers Length: 2119   
ASN2
ASN2 Customers Length: 825    
6, False, 2276.41550418085
MSN
MSN Customers Length: 6137    
ASN1
ASN1 Customers Length: 2149   
ASN2
ASN2 Customers Length: 778    
7, False, 1486.638022203007
MSN
MSN Customers Length: 6141    
ASN1
ASN1 Customers Length: 2238   

ASN2
ASN2 Customers Length: 810    
20, False, 648.9244987375656
MSN
MSN Customers Length: 6055    
ASN1
ASN1 Customers Length: 2200   
ASN2
ASN2 Customers Length: 768    
21, False, 543.3133333216192
MSN
MSN Customers Length: 6217    
ASN1
ASN1 Customers Length: 2223   
ASN2
ASN2 Customers Length: 835    
22, False, 746.527313077027
MSN
MSN Customers Length: 6050    
ASN1
ASN1 Customers Length: 2186   
ASN2
ASN2 Customers Length: 782    
23, False, 665.1544277118401
MSN
MSN Customers Length: 6121    
ASN1
ASN1 Customers Length: 2149   
ASN2
ASN2 Customers Length: 848    
24, False, 409.6002841283233
MSN
MSN Customers Length: 6241    
ASN1
ASN1 Customers Length: 2192   
ASN2
ASN2 Customers Length: 770    
25, False, 768.9445498481094
MSN
MSN Customers Length: 6039    
ASN1
ASN1 Customers Length: 2186   
ASN2
ASN2 Customers Length: 776    
26, False, 470.7065944881581
MSN
MSN Customers Length: 6143    
ASN1
ASN1 Customers Length: 2216   
ASN2
ASN2 Customers Length: 814    
27, False, 56

MSN
MSN Customers Length: 6126    
ASN1
ASN1 Customers Length: 2158   
ASN2
ASN2 Customers Length: 782    
40, False, 342.90530215211214
MSN
MSN Customers Length: 6019    
ASN1
ASN1 Customers Length: 2238   
ASN2
ASN2 Customers Length: 812    
41, False, 257.4906313607831
MSN
MSN Customers Length: 6023    
ASN1
ASN1 Customers Length: 2164   
ASN2
ASN2 Customers Length: 814    
42, False, 270.9022803920649
MSN
MSN Customers Length: 6138    
ASN1
ASN1 Customers Length: 2091   
ASN2
ASN2 Customers Length: 775    
43, False, 296.20146110116264
MSN
MSN Customers Length: 6113    
ASN1
ASN1 Customers Length: 2143   
ASN2
ASN2 Customers Length: 784    
44, False, 295.06173218092783
MSN
MSN Customers Length: 5966    
ASN1
ASN1 Customers Length: 2235   
ASN2
ASN2 Customers Length: 847    
45, False, 247.66487354169266
MSN
MSN Customers Length: 6253    
ASN1
ASN1 Customers Length: 2144   
ASN2
ASN2 Customers Length: 763    
46, False, 362.98448340135633
MSN
MSN Customers Length: 6154    
ASN1
ASN

ASN1
ASN1 Customers Length: 2196   
ASN2
ASN2 Customers Length: 803    
59, False, 265.3109457060355
MSN
MSN Customers Length: 6114    
ASN1
ASN1 Customers Length: 2171   
ASN2
ASN2 Customers Length: 816    
60, False, 208.3775047437643
MSN
MSN Customers Length: 6155    
ASN1
ASN1 Customers Length: 2111   
ASN2
ASN2 Customers Length: 849    
61, False, 249.2919911198025
MSN
MSN Customers Length: 6115    
ASN1
ASN1 Customers Length: 2187   
ASN2
ASN2 Customers Length: 772    
62, False, 262.42565911567345
MSN
MSN Customers Length: 6184    
ASN1
ASN1 Customers Length: 2106   
ASN2
ASN2 Customers Length: 809    
63, False, 344.6294191082323
MSN
MSN Customers Length: 6034    
ASN1
ASN1 Customers Length: 2130   
ASN2
ASN2 Customers Length: 774    
64, False, 201.54224592563185
MSN
MSN Customers Length: 6185    
ASN1
ASN1 Customers Length: 2145   
ASN2
ASN2 Customers Length: 866    
65, False, 148.3786997891726
MSN
MSN Customers Length: 6295    
ASN1
ASN1 Customers Length: 2256   
ASN2
ASN2 

ASN2
ASN2 Customers Length: 803    
78, False, 229.10167941534579
MSN
MSN Customers Length: 6115    
ASN1
ASN1 Customers Length: 2162   
ASN2
ASN2 Customers Length: 797    
79, False, 175.36691502519665
MSN
MSN Customers Length: 6173    
ASN1
ASN1 Customers Length: 2159   
ASN2
ASN2 Customers Length: 769    
80, False, 234.81560528412706
MSN
MSN Customers Length: 6032    
ASN1
ASN1 Customers Length: 2248   
ASN2
ASN2 Customers Length: 763    
81, False, 168.46952747676707
MSN
MSN Customers Length: 6156    
ASN1
ASN1 Customers Length: 2225   
ASN2
ASN2 Customers Length: 803    
82, False, 159.40335467684358
MSN
MSN Customers Length: 6211    
ASN1
ASN1 Customers Length: 2167   
ASN2
ASN2 Customers Length: 825    
83, False, 200.79153697882285
MSN
MSN Customers Length: 6147    
ASN1
ASN1 Customers Length: 2173   
ASN2
ASN2 Customers Length: 761    
84, False, 189.76428354832188
MSN
MSN Customers Length: 6104    
ASN1
ASN1 Customers Length: 2137   
ASN2
ASN2 Customers Length: 771    
85, F

97, False, 117.74634612192108
MSN
MSN Customers Length: 6105    
ASN1
ASN1 Customers Length: 2121   
ASN2
ASN2 Customers Length: 814    
98, False, 117.3391428433273
MSN
MSN Customers Length: 5931    
ASN1
ASN1 Customers Length: 2225   
ASN2
ASN2 Customers Length: 799    
99, False, 97.45367301171078
MSN
MSN Customers Length: 6154    
ASN1
ASN1 Customers Length: 2176   
ASN2
ASN2 Customers Length: 838    
100, True, 155.51894786907187


In [16]:
def bootstrap(data, f_statistic, draws):
    """ Calculates the bootstrap mse of a statistic of choice
    
    Keywords:
        data (array): data array.
        f_statistic: function handle calculating the statistic of interest.
        draws (int): number of bootstrap draws.
    
    Returns:
        mse (float): mean square error of the statistic of interest.
    """
    theta = f_statistic(data)
    se = np.zeros((draws, ))
    for r in np.arange(draws):
        data_draw = np.random.choice(data, size=data.shape[0], replace=True)
        theta_emp = f_statistic(data_draw)
        se[r] = (theta_emp -  theta)**2
    mse = se.mean()
    return mse

In [18]:
f_mean = lambda data: np.mean(data)
for group in groups:
    
    queue_all = final_statistics[group][parameter_queue_all]
    bootstrap_mean = bootstrap(np.array(queue_all), f_mean, t)
    empirical_mean = np.mean(queue_all)
    print('{:<20} {:<20} {:<20}'.format(f'Group: {group}', f'Bootstrap: {bootstrap_mean:.2f}', f'Empirical: {empirical_mean}'))

Group: MSN           Bootstrap: 2.84      Empirical: 145.67782252018927
Group: ASN1          Bootstrap: 0.00      Empirical: 6.654184107537726
Group: ASN2          Bootstrap: 0.00      Empirical: 7.076584321113865
Group: 1             Bootstrap: 0.87      Empirical: 83.94740089232346
Group: 2             Bootstrap: 1.11      Empirical: 97.46478003252997
Group: 3             Bootstrap: 2.22      Empirical: 118.499562485941


# Variance Reduction

## Antithetic Runs

In [ ]:
precision = 0.1
t = 0

groups = [
    ServerIDs.msn.value,
    ServerIDs.asn_1.value,
    ServerIDs.asn_2.value,
    Groups.Group_1.value,
    Groups.Group_2.value,
    Groups.Group_3.value
]

final_statistics = dict()


parameter_queue = 'mean'
parameter_queue_mean = f'{parameter_queue}_queue_mean'
parameter_queue_var = f'{parameter_queue}_queue_var'
parameter_queue_mean_all = f'{parameter_queue}_queue_mean_all'
parameter_queue_var_all = f'{parameter_queue}_queue_var_all'
parameter_queue_all = f'{parameter_queue}_all'

for group in groups:
    final_statistics[group] = dict()

for group in groups:
    final_statistics[group][parameter_queue_mean] = 0
    final_statistics[group][parameter_queue_var] = 0
    final_statistics[group][parameter_queue_mean_all] = []
    final_statistics[group][parameter_queue_var_all] = []
    final_statistics[group][parameter_queue_all] = []

converged = False

# We converge if and only if
# for every group \in Groups
# and for every server \in ServerIds
# we have
# t >= 100 and np.sqrt(var / t) < precision
while not converged: 
    t += 1
    
    # Run simulation (independent)
    u = np.random.rand(12, 15000)
    results = handle_requests(u)
    
        
    # Run simulation (antithetic)
    u = 1 - u
    results_antithetic = handle_requests(u)

    # Collect statistics
    statistics = get_statistics(results)
    statistics_antithetic = get_statistics(results_antithetic)
    for group in groups:
        
        par_queue = (statistics[group][parameter_queue] + statistics_antithetic[group][parameter_queue]) / 2
        mean, var = moving_mean_var(
            par_queue,\
            final_statistics[group][parameter_queue_mean],\
            final_statistics[group][parameter_queue_var],\
            t
        )
        
        final_statistics[group][parameter_queue_all].append(par_queue)
        final_statistics[group][parameter_queue_mean_all].append(mean)
        final_statistics[group][parameter_queue_var_all].append(var)
        
    # Check if necessary precision reached
    converged = True
    for group in groups:
        var = final_statistics[group][parameter_queue_var_all][-1]
        converged = converged and (t >= 100 and np.sqrt(var / t) < precision)
    print(f'{t}, {converged}, {var}')